# Main

Using the notebook will allow you to load the model once and use it as many times as you
want. Also makes the code more resilient to errors, such as a bad path.

In [1]:
from pathlib import Path

INFO_TAG = "persuasive_essays_paragraph"
BASE_DATA = Path("data")

SOURCE_LANGUAGE = "english"
TARGET_LANGUAGE = "spanish"

# Corpus Projection
CORPUS = BASE_DATA / "corpus" / INFO_TAG
PROCESSED_CORPUS = BASE_DATA / "parsed_to_conll" / INFO_TAG
SENTENCE_ALIGN = BASE_DATA / 'sentence_alignment' / INFO_TAG
BIDIRECTIONAL_ALIGN = BASE_DATA / 'bidirectional_alignment' / INFO_TAG
PROJECTION = BASE_DATA / 'projection' / INFO_TAG

# Link Prediction
TO_PROCESS = BASE_DATA / "to_process" / INFO_TAG
SEGMENTER = BASE_DATA / "segmenter_processed" / INFO_TAG
LINK_PREDICTION = BASE_DATA / 'link_prediction_processed' / INFO_TAG

# Export to Brat
BRAT = Path("brat", "data", INFO_TAG)


# Corpus Projection

In [ ]:
from pipelines.corpus_pipelines import full_corpus_processing_pipeline

from aligner.aligner import AwesomeAlignAligner as Aligner
from corpus_parser.unified_parser import UnifiedParser as Parser
from projector.projector import CrossLingualAnnotationProjector as Projector
from sentence_aligner.sentence_aligner import SentenceAligner
from sentence_aligner.translator import GoogleDeepTranslator as Translator

full_corpus_processing_pipeline(
    corpus_dir=CORPUS,
    standard_corpus_dest_dir=PROCESSED_CORPUS,
    sentence_alignment_dest_dir=SENTENCE_ALIGN,
    bidirectional_alignment_dest_dir=BIDIRECTIONAL_ALIGN,
    projection_dest_dir=PROJECTION,
    corpus_parser=Parser(),
    sentence_aligner=SentenceAligner(Translator()),
    aligner=Aligner(),
    projector=Projector(),
    source_language=SOURCE_LANGUAGE,
    target_language=TARGET_LANGUAGE
)

# Link Prediction

In [2]:
from pipelines.segmenter_pipelines import perform_full_inference_pipeline, perform_segmentation_pipeline, perform_link_prediction_pipeline


In [ ]:
from segmenter.tf_segmenter import TensorflowArgumentSegmenter as Segmenter

segmenter = Segmenter(INFO_TAG, TARGET_LANGUAGE)


In [3]:
from link_prediction.tf_link_predictor import TensorflowLinkPredictor as LinkPredictor

link_predictor = LinkPredictor(INFO_TAG, TARGET_LANGUAGE)

dev relations 652
dev source argumentative units 326
dev target argumentative units 144
test relations 1618
test source argumentative units 809
test target argumentative units 365
train relations 5392
train source argumentative units 2696
train target argumentative units 1198
Vocab size 8958
Relation tags ['supports', 'attacks', 'supports_Inverse', 'attacks_Inverse']
Proposition tags ['Claim', 'MajorClaim', 'Premise']
max_size_prop 70
max_amount_doc 20
[('supports', 'Premise', 'Claim'), ('', 'Premise', 'Premise'), ('supports', 'Premise', 'Premise')]
3


In [4]:

# perform_segmentation_pipeline(
#     segmenter=segmenter,
#     source_dir=TO_PROCESS,
#     destination_dir=SEGMENTER,
# )

perform_link_prediction_pipeline(
    link_predictor=link_predictor,
    source_dir=SEGMENTER,
    destination_dir=LINK_PREDICTION,
    source_language=TARGET_LANGUAGE
)

# perform_full_inference_pipeline(
#     segmenter=segmenter,
#     link_predictor=link_predictor,
#     source_dir=TO_PROCESS,
#     segmenter_destination_dir=SEGMENTER,
#     destination_dir=LINK_PREDICTION,
#     source_language=TARGET_LANGUAGE
# )

# Export to Brat

In [11]:
from corpus_parser.brat_parser import BratParser
from corpus_parser.conll_parser import ConllParser

dataframes_dict = ConllParser(bioes=True).parse_dir(LINK_PREDICTION)

BratParser().export_from_dataframes(BRAT, dataframes_dict)
